# Setup

In [1]:
path = "data/join"
topic = "立法方式保障"

In [2]:
from __future__ import division, print_function
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import jieba
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout, BatchNormalization, Conv1D, ZeroPadding1D, MaxPooling1D
import os, math, re, pickle

jieba.set_dictionary(os.path.join("data", "dict.txt.big"))

Using TensorFlow backend.


In [3]:
_data = None

def load_data():
    global _data
    if _data is None:
        _data = pd.read_csv(os.path.join(path, topic + "-sentences.csv"))
    return _data

Build dictionary for word embedding.

In [4]:
dictionary_path = os.path.join(path, "dictionary.pkl")

clean_phrase_re = re.compile(r"[ ]+")

def cleaned_phrase(phrases):
    for ph in phrases:
        ph = clean_phrase_re.sub("", ph)
        if ph != "":
            yield ph

def create_dictionary(*data):
    phrases = {}
    for d in data:
        for sentence in d:
            for ph in cleaned_phrase(jieba.cut(str(sentence))):
                phrases[ph] = True
    with open(os.path.join(path, "dictionary.txt"), "w") as fh:
        fh.writelines([ ph + "\n" for ph in phrases.keys() ])
    !cd $path; mkdir -p models; ln ../fasttext/wiki.zh.bin models/wiki.zh.bin
    !cd $path; ~/bin/fasttext print-word-vectors models/wiki.zh.bin < dictionary.txt > dictionary.vec
    dictionary = pd.read_csv(os.path.join(path, "dictionary.vec"), 
                             delim_whitespace=True, engine="python", header=None, index_col=0)
    with open(dictionary_path, "wb") as fh:
        pickle.dump([{ ph: i for i, ph in enumerate(dictionary.index) }, dictionary], fh)

def load_dictionary():
    with open(dictionary_path, "rb") as fh:
        [ dict_index, dictionary ] = pickle.load(fh)
        return dict_index, dictionary
    
if not os.path.exists(dictionary_path):
    data = load_data()
    create_dictionary(data.sentence)

dict_index, dictionary = load_dictionary()
phrases_n = len(dictionary)
latent_n = len(dictionary.columns)

Examine phrase length.

In [5]:
#data = load_data()
#stat = np.frompyfunc(lambda s: len(jieba.lcut(str(s))), 1, 1)(data.sentence.values)
#(stat.min(), stat.max(), stat.mean(), stat.std())

Encode lables and embed phrases.

In [14]:
# phrase-length (min, max, mean, std) = (1, 1096, 17.385437090122373, 20.549680891647231)
input_length = 20

data_path = os.path.join(path, "data.pkl")

orid_index = { "O": 0, "R": 1, "I": 2, "D": 3 }

def get_label(df):
    labels = np.zeros((len(df), 4))
    for i, l in enumerate(df["orid"]):
        j = orid_index[l]
        labels[i, j] = 1
    return labels

def get_text(df):
    texts = np.zeros((len(df), input_length))
    for i, text in enumerate(df["sentence"].values):
        for j, ph in enumerate(cleaned_phrase(jieba.lcut(str(text)))):
            if j >= input_length:
                break
            if ph in dict_index:
                texts[i, j] = dict_index[ph]
    return texts

if not os.path.exists(data_path):
    data = load_data()
    data = data[data["orid"].notnull()]
    mask = np.random.random(len(data)) > 0.1
    train, valid = data[mask], data[~mask]
    train_x, train_y = get_text(train), get_label(train)
    valid_x, valid_y = get_text(valid), get_label(valid)
    
    with open(data_path, "wb") as fh:
        pickle.dump([(train_x, train_y), (valid_x, valid_y)], fh)
else:
    with open(data_path, "rb") as fh:
        [(train_x, train_y), (valid_x, valid_y)] = pickle.load(fh)

# Simple CNN

In [35]:
batch_size = 64

def simple_cnn_model():
    model = Sequential()
    model.add(Embedding(phrases_n, latent_n, input_length=input_length, weights=[dictionary], trainable=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 3, border_mode="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(100, activation="relu"))
    model.add(Dropout(0.7))
    model.add(Dense(4, activation="softmax"))
    return model
    
simple_cnn = simple_cnn_model()
simple_cnn.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])
simple_cnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_13 (Embedding)         (None, 20, 300)       9825000     embedding_input_13[0][0]         
____________________________________________________________________________________________________
batchnormalization_25 (BatchNorm (None, 20, 300)       1200        embedding_13[0][0]               
____________________________________________________________________________________________________
dropout_37 (Dropout)             (None, 20, 300)       0           batchnormalization_25[0][0]      
____________________________________________________________________________________________________
convolution1d_13 (Convolution1D) (None, 20, 64)        57664       dropout_37[0][0]                 
___________________________________________________________________________________________

In [36]:
def train_simple_cnn(lr=None, epoch=1, full=False):
    if lr is not None:
        simple_cnn.optimizer.lr = lr
    if full:
        simple_cnn.layers[0].trainable = True
    simple_cnn.fit(train_x, train_y, nb_epoch=epoch, batch_size=batch_size, validation_data=(valid_x, valid_y))
    
train_simple_cnn(1e-4)
train_simple_cnn(1e-1, 4)
train_simple_cnn(1e-2, 8)
train_simple_cnn(1e-3, 8)
train_simple_cnn(1e-4, 2, full=True)

Train on 181 samples, validate on 17 samples
Epoch 1/1
181/181 [==============================] - 1s - loss: 3.1476 - acc: 0.2210 - val_loss: 1.4112 - val_acc: 0.1176
Train on 181 samples, validate on 17 samples
Epoch 1/4
181/181 [==============================] - 0s - loss: 2.7716 - acc: 0.2818 - val_loss: 1.4077 - val_acc: 0.0588
Epoch 2/4
181/181 [==============================] - 0s - loss: 2.7141 - acc: 0.2431 - val_loss: 1.4005 - val_acc: 0.0588
Epoch 3/4
181/181 [==============================] - 0s - loss: 2.4590 - acc: 0.3536 - val_loss: 1.3949 - val_acc: 0.0588
Epoch 4/4
181/181 [==============================] - 0s - loss: 2.1359 - acc: 0.3867 - val_loss: 1.3902 - val_acc: 0.1765
Train on 181 samples, validate on 17 samples
Epoch 1/12
181/181 [==============================] - 0s - loss: 2.0767 - acc: 0.4254 - val_loss: 1.3849 - val_acc: 0.2353
Epoch 2/12
181/181 [==============================] - 0s - loss: 1.7785 - acc: 0.4696 - val_loss: 1.3797 - val_acc: 0.2941
Epoch 3/1

In [12]:
simple_cnn.save_weights(os.path.join(path, "models", "simple_cnn.h5"))

# Evaluate model

In [13]:
model = simple_cnn_model()
model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.load_weights(os.path.join(path, "models", "simple_cnn.h5"))

In [ ]:
data = load_data()
data = data[data["orid"].isnull()]
testing_x = get_text(data)
pred = model.predict(testing, batchsize=batchsize)
